## Applying the bias scan tool on a loan approval classifier
In this notebook, the bias scan tool is applied on a XGBoost loan approval classifier. The bias scan tool is based on an implementation of the k-means Hierarchical Bias Aware Clustering (HBAC) method\*. The python script `./helper_functions.py` contains functions that execute the bias scan. A conceptual description how the bias scan works, including the rationale why k-means is chosen as a clustering algorithm and paramater choices, can be found in the [bias scan report](https://github.com/NGO-Algorithm-Audit/Bias_scan/blob/master/Bias_scan_tool_report.pdf) (technical documentation).

An XGBoost classifier is trained to make predictions on the German Credit\*\* data set. Details on pre-processing steps performed on this dataset and the trained XGBoost classifier are provided in the `../case_studies/Loan_approval_classifier/GermanCredit_classifier.ipynb` notebook.

\* Misztal-Radecka, Indurkya, *Information Processing and Management*. Bias-Aware Hierarchical Clustering for detecting the discriminated groups of users in recommendation systems (2021).

\*\* the original dataset, in the form provided by Prof. Hofmann, contains categorical/symbolic attributes which is contained in the file "german.data" [[link to dataset]](https://archive.ics.uci.edu/ml/datasets/Statlog+%28German+Credit+Data%29)

### Overview of the notebook
1. Load data and pre-processing
2. Bias scan using k-means clustering
3. Clustering results
4. Statistical testing of inter-cluster difference 

### Load libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Markdown, display

# helper functions
from helper_functions import *

# sklearn
from sklearn.preprocessing import StandardScaler

# welch's t-test
import scipy.stats as stats

### 1. Load data and pre-processing
#### Load data

In [ ]:
path = '../classifiers/Loan_approval_classifier/pred_XGBoost.csv'

# read data
entire_dataset = pd.read_csv(path)
entire_dataset.head()

#### Data cleaning

In [ ]:
features = entire_dataset.drop(['predicted_class', 'true_class', 'errors'], axis=1)
features.head()

#### Scaling data

In [ ]:
full_data = init_GermanCredit_dataset(entire_dataset,features)
full_data.head()

### 2. Bias scan using k-means clustering
Clustering algorithms parameters:

In [ ]:
clustering_paramaters = {
    "n_clusters": 3,
    "init": "k-means++",
    "n_init": 20,
    "max_iter": 300,
    "random_state": 10,
}

Specify:
- Minimal splittable cluster size
- Minimal acceptable cluster size

In [ ]:
# minimal splittable cluster size
split_cluster_size = round(0.05 * len(full_data))
print("minimal splittable cluster size: ", split_cluster_size)

# minimal acceptable cluster size
acc_cluster_size = round(0.03 * len(full_data))
print("minimal splitacceptabletable cluster size: ", acc_cluster_size)

Performing bias scan using helper functions.

In [ ]:
metric = 'accuracy'
iterations_max = 20
x = 0 # initial cluster number
initial_bias = 0
variance_list = []
average_accuracy = accuracy(full_data, metric)
minimal_splittable_cluster_size = 18
minimal_acceptable_cluster_size = 10
print("average_accuracy is: ", average_accuracy) 

for i in range(1, iterations_max):
    if i != 1:
        variance_list.append(calculate_variance(full_data,metric)) 
    full_data['new_clusters'] = -1
    candidate_cluster = full_data.loc[full_data['clusters'] == x] 

    if len(candidate_cluster) < minimal_splittable_cluster_size:
        x = get_random_cluster(full_data['clusters'])
        continue
    
    # Apply Kmeans 
    kmeans_algo = KMeans(**clustering_paramaters).fit(candidate_cluster.drop(['clusters', 'new_clusters', 'predicted_class', 'true_class', 'errors'], axis=1))
    
    candidate_cluster['new_clusters'] = pd.DataFrame(kmeans_algo.predict(candidate_cluster.drop(['clusters', 'new_clusters', 'predicted_class', 'true_class', 'errors'], axis=1)),index=candidate_cluster.index) 
    full_data['new_clusters'] = candidate_cluster['new_clusters'].combine_first(full_data['new_clusters'])

    max_abs_bias = get_max_negative_bias(full_data,metric) # was get_max_abs_bias, but now it only finds the discriminated clusters
    min_new_size = get_min_cluster_size(full_data)
    
    if (max_abs_bias <= initial_bias) & (min_new_size > minimal_acceptable_cluster_size): #abs: >
        # add new cluster
        n_cluster = max(full_data['clusters'])
        full_data['clusters'][full_data['new_clusters'] == 1] =  n_cluster + 1
        
        x = get_next_cluster(full_data)
        initial_bias = max_abs_bias
    else:
        x = get_random_cluster(full_data['clusters'])
        
print('done')

### 3. Analysing clustering results
Identifying cluster with most negative bias.

In [ ]:
c = get_max_bias_cluster(full_data, metric)
max_bias = round(bias_acc(full_data, metric, c, "clusters"), 2)
highest_biased_cluster = full_data[full_data['clusters']==c]
print(f"cluster {c} has the highest negative bias: " + str(max_bias))
print("#elements in highest biased cluster:", len(highest_biased_cluster))

Accuracy of classifier on full dataset and on cluster with highest discrimination bias.

In [ ]:
accuracy_full_data = accuracy(full_data, metric)
most_biased_cluster_kmeans_aware = bias_acc(full_data, metric, c, 'clusters')
full_data[full_data['clusters']==c]

print('General accuracy of classifier on this dataset:', accuracy_full_data)

Select and print discriminated cluser.

In [ ]:
c=c
discriminated_cluster = full_data[full_data['clusters']==c]
print('Number of instances in discriminated cluster:', len(discriminated_cluster))
print('Number of errors in discriminated clusters: ', len(discriminated_cluster.loc[discriminated_cluster['errors']==1]))
discriminated_cluster

### 4. Statistical testing of inter-cluster difference 
Compute difference between cluster with highest discrimination bias and rest of dataset.

In [ ]:
not_discriminated = full_data[full_data['clusters']!=c]
difference = (discriminated_cluster.mean()) - (not_discriminated.mean()) 
diff_dict = difference.to_dict()
difference

Unscaling the data. Print summary of cluster with highest discrimination bias and the rest of data.

In [ ]:
# unscaling the discriminated cluster
unscaled_discriminated = full_data.loc[discriminated_cluster.index, :]

# unscaled other data
unscaled_remaining = full_data.drop(discriminated_cluster.index)

display(unscaled_remaining.describe())
display(unscaled_discriminated.describe())

print(discriminated_cluster.index)

In [ ]:
unscaled_discriminated

####  Test to check statistical significance of inter-cluster difference (per feature)
Applying a Welch’s two-samples t-test for unequal variances to examine whether the differences in means for each feature are statistically significant and store results in a dictionary.

In [ ]:
welch_dict = {}

features = [col for col in full_data.columns.tolist() if col not in ['scaled_errors','predicted_class','true_class','errors','clusters','new_clusters']]
    
for i in features:
    welch_i = stats.ttest_ind(unscaled_discriminated[i], unscaled_remaining[i], equal_var=False)
    
    # attach to dictionary
    welch_dict[i] = welch_i.pvalue
    
welch_dict

#### p-values
A small p-value (p < 0.05) indicates that it is unlikely to observe inter-cluster difference due to chance. Sort difference on statistical significance (p-value).

In [ ]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option("display.max_rows", None, "display.max_columns", None)
cluster_analysis_df = pd.DataFrame([diff_dict, welch_dict]).T
cluster_analysis_df.columns = ['difference','p-value']
cluster_analysis_df = cluster_analysis_df.sort_values('p-value')
cluster_analysis_df

#### Conclusion
Suspected negative bias by the classifier on the basis of (p<0.05):
- Housing (`housing=A151,A152,A153`);
- Property real estate or unknown (`property=A121,A124`);
- Other debtors (`other_debtors=A101,A102,A103`);
- Employment (`employment=A71`);
- Installment plans (`installment_plans=A142,A=143`);
- Foreign worker (`foreign_worker=A201,A202`);
- Month (`month`);
- Investment as percentage of annual income (`investment_as_income_percentage`);
- Gender (`sex`);
- Present residence since (`residence since`);
- Age in years (`age`)
- Type of property (`property=A42,A44,A45`);
- Job type (`skill_level=A174`);
- Amount of credit requested (`credit_amount`).

This means that loan applicants with the above features on average are significantly less often approved by the XGBoost classifier. An exact description of the features can be found [here](https://archive.ics.uci.edu/ml/datasets/Statlog+%28German+Credit+Data%29).

#### What's next?
Qualitative assessment with the help of subject matter experts to verify the measured quantitaive disparities. Additionally, sensitivity testing would be beneficial to shed light into the robustness of the bias scan tool. 